## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for Logical Observation Identifiers Names and Codes (LOINC)

- **Model**: `loinc_resolver_pipeline`
- **Model Description**: This pipeline extracts `Test` entities from clinical texts and maps them to their corresponding Logical Observation Identifiers Names and Codes (LOINC) codes using `sbiobert_base_cased_mli` Sentence Bert Embeddings.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [17]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [18]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [19]:
model_name = "loinc-resolver-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [20]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

INFO:sagemaker:Creating model with name: loinc-resolver-pipeline-2024-06-10-12-19-31-977
INFO:sagemaker:Creating endpoint-config with name loinc-resolver-pipeline
INFO:sagemaker:Creating endpoint with name loinc-resolver-pipeline


----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [21]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [22]:
docs = [
    "The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.", 
    "Final diagnosis:  : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2)   Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.",
]


sample_text = """A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.
  She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension
  for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that
  includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are
  within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3"""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [23]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,loinc_code,loinc_resolution,loinc_confidence
0,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that\n includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are\n within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3",Vital signs,448,458,Test,0.87475,8716-3,Vital signs [Vital signs],0.9796
1,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that\n includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are\n within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3",A physical examination,488,509,Test,0.91120005,LP7801-6,Physical exam [Physical exam],0.2517
2,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that\n includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are\n within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3",Laboratory studies,528,545,Test,0.70070004,26436-6,Laboratory studies [Laboratory studies],0.9922
3,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that\n includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are\n within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3",Hemoglobin,567,576,Test,0.9995,14775-1,Hemoglobin [Hemoglobin],0.8477
4,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married and lives with her husband. She eats a balanced diet that\n includes chicken, fish, pork, fruits, and vegetables. She rarely drinks alcohol and denies tobacco use. Vital signs are\n within normal limits. A physical examination is unremarkable. Laboratory studies show the following: Hemoglobin: 9.8g/dL, Hematocrit: 32%, Mean Corpuscular Volume: 110 μm3",Hematocrit,588,597,Test,0.9986,11151-8,Hematocrit [Hematocrit],0.9867
5,"A 65-year-old woman presents to the office with generalized fatigue for the last 4 months.\n She used to walk 1 mile each evening but now gets tired after 1-2 blocks. She has a history of Crohn disease and hypertension\n for which she receives appropriate medications. She is married

#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [24]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,loinc_code,loinc_resolution,loinc_confidence
0,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",BMI),90,93,Test,0.698,39156-5,BMI [Body mass index],0.4923
1,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",aspartate aminotransferase,110,135,Test,0.8905,14409-7,Aspartate aminotransferase [Aspartate aminotransferase],0.9928
2,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",alanine aminotransferase,145,168,Test,0.825,16324-6,Alanine aminotransferase [Alanine aminotransferase],0.9868
3,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,the peripheral blood specimen,43,71,Test,0.69299996,LP7072-4,Blood peripheral [Blood peripheral],0.4696
4,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,a polymerase chain reaction,77,103,Test,0.73675,LP89236-1,RNA polymerase I [RNA polymerase I],0.0792
5,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,PCR)-based assay,106,121,Test,0.75715,13590-5,aPCR PPP [Activated protein C resistance],0.0693


### JSON Lines

In [25]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [26]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "BMI)", "begin": 90, "end": 93, "ner_label": "Test", "ner_confidence": "0.698", "loinc_code": "39156-5", "loinc_resolution": "BMI [Body mass index]", "loinc_confidence": "0.4923"}, {"ner_chunk": "aspartate aminotransferase", "begin": 110, "end": 135, "ner_label": "Test", "ner_confidence": "0.8905", "loinc_code": "14409-7", "loinc_resolution": "Aspartate aminotransferase [Aspartate aminotransferase]", "loinc_confidence": "0.9928"}, {"ner_chunk": "alanine aminotransferase", "begin": 145, "end": 168, "ner_label": "Test", "ner_confidence": "0.825", "loinc_code": "16324-6", "loinc_resolution": "Alanine aminotransferase [Alanine aminotransferase]", "loinc_confidence": "0.9868"}]}
{"predictions": [{"ner_chunk": "the peripheral blood specimen", "begin": 43, "end": 71, "ner_label": "Test", "ner_confidence": "0.69299996", "loinc_code": "LP7072-4", "loinc_resolution": "Blood peripheral [Blood peripheral]", "loinc_confidence": "0.4696"}, {"ner_chunk": "a polymerase c

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [27]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

INFO:sagemaker:Deleting endpoint with name: loinc-resolver-pipeline
INFO:sagemaker:Deleting endpoint configuration with name: loinc-resolver-pipeline


## 3. Batch inference

In [28]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [29]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [31]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [32]:
process_s3_json_output_and_save(validation_json_file_name)

,document,ner_chunk,begin,end,ner_label,ner_confidence,loinc_code,loinc_resolution,loinc_confidence
0,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",BMI),90,93,Test,0.698,39156-5,BMI [Body mass index],0.4923
1,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",aspartate aminotransferase,110,135,Test,0.8905,14409-7,Aspartate aminotransferase [Aspartate aminotransferase],0.9928
2,"The patient is a 22-year-old female with a history of obesity. She has a Body mass index (BMI) of 33.5 kg/m2, aspartate aminotransferase 64, and alanine aminotransferase 126.",alanine aminotransferase,145,168,Test,0.825,16324-6,Alanine aminotransferase [Alanine aminotransferase],0.9868
3,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,the peripheral blood specimen,43,71,Test,0.69299996,LP7072-4,Blood peripheral [Blood peripheral],0.4696
4,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,a polymerase chain reaction,77,103,Test,0.73675,LP89236-1,RNA polymerase I [RNA polymerase I],0.0792
5,Final diagnosis: : DNA was extracted from the peripheral blood specimen and a polymerase chain reaction (PCR)-based assay performed that is designed to detect the presence of two separate mutations in the FLT3 gene: (1) within a susceptible region that includes coding sequence for the intracellular juxtamembrane domain and (2) Negative. Neither expansion of the region susceptible to nor changes consistent with mutation of the codon for ASP835 were identified.,PCR)-based assay,106,121,Test,0.75715,13590-5,aPCR PPP [Activated protein C resistance],0.0693


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [34]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [35]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "BMI)", "begin": 90, "end": 93, "ner_label": "Test", "ner_confidence": "0.698", "loinc_code": "39156-5", "loinc_resolution": "BMI [Body mass index]", "loinc_confidence": "0.4923"}, {"ner_chunk": "aspartate aminotransferase", "begin": 110, "end": 135, "ner_label": "Test", "ner_confidence": "0.8905", "loinc_code": "14409-7", "loinc_resolution": "Aspartate aminotransferase [Aspartate aminotransferase]", "loinc_confidence": "0.9928"}, {"ner_chunk": "alanine aminotransferase", "begin": 145, "end": 168, "ner_label": "Test", "ner_confidence": "0.825", "loinc_code": "16324-6", "loinc_resolution": "Alanine aminotransferase [Alanine aminotransferase]", "loinc_confidence": "0.9868"}]}
{"predictions": [{"ner_chunk": "the peripheral blood specimen", "begin": 43, "end": 71, "ner_label": "Test", "ner_confidence": "0.69299996", "loinc_code": "LP7072-4", "loinc_resolution": "Blood peripheral [Blood peripheral]", "loinc_confidence": "0.4696"}, {"ner_chunk": "a polymerase c

In [36]:
model.delete_model()

INFO:sagemaker:Deleting model with name: loinc-resolver-pipeline-2024-06-10-12-35-00-545


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

